In [5]:
from dataset import CustomSyntheticDataset
from neuralNetTraining import NeuralNetwork
import torch
from torch import nn
from torchvision.transforms import Lambda
from torch.utils.data import Dataset, DataLoader


In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
train_dataset = CustomSyntheticDataset(target_transform=Lambda(lambda y: torch.zeros(11, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),datasetSize=40000)
test_dataset = CustomSyntheticDataset(target_transform=Lambda(lambda y: torch.zeros(11, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),datasetSize=1000)


# train_dataset = CustomSyntheticDataset(datasetSize=10000)
# test_dataset = CustomSyntheticDataset(datasetSize=1000)
train_dataloader = DataLoader(train_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

model = NeuralNetwork()


learning_rate = 1e-3
batch_size = 64




balancedLoss = False
if(balancedLoss):
    classDist = train_dataset.empiricalWeight()
    weights = torch.zeros(train_dataset.distCount)
    for i in range(train_dataset.distCount):
        if classDist[i] != 0:
            weights[i] = 1/classDist[i]

    loss_fn = nn.CrossEntropyLoss(weight=weights)
else:
    # Initialize the loss function
    # this loss function includes softmax
    loss_fn = nn.CrossEntropyLoss()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


epochs =20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.466428  [    0/40000]
Test Error: 
 Accuracy: 91.0%, Avg loss: 1.018105 

Epoch 2
-------------------------------
loss: 0.935243  [    0/40000]
Test Error: 
 Accuracy: 91.0%, Avg loss: 0.784215 

Epoch 3
-------------------------------
loss: 0.709889  [    0/40000]
Test Error: 
 Accuracy: 91.0%, Avg loss: 0.690876 

Epoch 4
-------------------------------
loss: 0.642598  [    0/40000]
Test Error: 
 Accuracy: 91.0%, Avg loss: 0.612382 

Epoch 5
-------------------------------
loss: 0.584670  [    0/40000]
Test Error: 
 Accuracy: 91.0%, Avg loss: 0.541607 

Epoch 6
-------------------------------
loss: 0.528624  [    0/40000]
Test Error: 
 Accuracy: 91.0%, Avg loss: 0.479590 

Epoch 7
-------------------------------
loss: 0.474738  [    0/40000]
Test Error: 
 Accuracy: 91.0%, Avg loss: 0.426430 

Epoch 8
-------------------------------
loss: 0.426760  [    0/40000]
Test Error: 
 Accuracy: 91.0%, Avg loss: 0.382078 

Epoch 9
----------------